## Importar librerías

In [7]:
import os
import glob
import pandas as pd
from genetic_algorithm import GeneticAlgorithmCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import KernelPCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
import xgboost as xgb
from concurrent.futures import ThreadPoolExecutor, wait
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

## Cargar y preparar datos

In [8]:
def create_training_dataset(df, trade_type):
    df = df.drop_duplicates()
    # Filtrar las operaciones del tipo especificado y con profit != 0
    df_trade = df[(df['type'] == trade_type) & (df['profit'] != 0)].copy()
    # Añadir la columna 'Target' basada en el profit
    df_trade['target'] = df_trade['profit'].apply(lambda x: 1 if x > 0 else 0)
    # Eliminar posibles missings
    if(df_trade.isna().values.any()):
        df_trade = df_trade.dropna()
    # Separar en ganadoras y perdedoras
    df_winning = df_trade[df_trade['profit'] > 0]
    df_losing = df_trade[df_trade['profit'] < 0]
    # Debug info
    n_winning = len(df_trade[df_trade['target']==1])
    n_losing = len(df_trade[df_trade['target']==0])
    print(f"Tipo de operación: {'Buy' if trade_type == 1 else 'Sell'}")
    print(f"Número de características: {df.iloc[:, :-2].shape[1]}")
    print(f"Total Ganadoras: {n_winning}")
    print(f"Total Perdedoras: {n_losing}")
    # Verificar que hay suficientes datos
    if n_winning == 0 or n_losing == 0:
        print(f"No hay suficientes datos para {'compras' if trade_type == 1 else 'ventas'} para entrenar el modelo.")
        return False
    # Equilibrar las clases
    if n_winning <= n_losing:
        n_samples_per_class = n_winning
        # Seleccionar todas las ganadoras
        selected_winning = df_winning.copy()
        # Ordenar las perdedoras por pérdida de mayor a menor (menor profit a mayor)
        df_losing_sorted = df_losing.sort_values(by='profit', ascending=True)
        # Seleccionar las perdedoras con mayor pérdida
        selected_losing = df_losing_sorted.head(n_samples_per_class)
    else:
        n_samples_per_class = n_losing
        # Seleccionar todas las perdedoras
        selected_losing = df_losing.copy()
        # Ordenar las ganadoras por profit de mayor a menor
        df_winning_sorted = df_winning.sort_values(by='profit', ascending=False)
        # Seleccionar las ganadoras con mayor profit
        selected_winning = df_winning_sorted.head(n_samples_per_class)
    print(f"Se seleccionarán {n_samples_per_class} muestras por clase.")
    print("___________________________________")
    # Combinar las muestras seleccionadas
    df_training = pd.concat([selected_winning, selected_losing], ignore_index=True)
    # Seleccionar las columnas necesarias (todas menos las dos últimas para el conjunto principal)
    feature_columns = df.columns[:-3]
    df_training = df_training[feature_columns.tolist() + ['target']]
    # Preparación de los datos de entrenamiento
    X_train = df_training.drop(columns='target').values
    y_train = df_training['target'].values
    # return dataframe
    return X_train, y_train, df_training

# Cargar el archivo CSV
file_folder = r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/"
file_pattern = os.path.join(file_folder, 'training_dataset_*.csv')
df_file_path = glob.glob(file_pattern)
df = pd.read_csv(df_file_path[0])
# Obtener datasets de entrenamiento
X_buy, y_buy, df_buy = create_training_dataset(df, 1)
X_sell, y_sell, df_sell = create_training_dataset(df, -1)

Tipo de operación: Buy
Número de características: 67
Total Ganadoras: 2457
Total Perdedoras: 2095
Se seleccionarán 2095 muestras por clase.
___________________________________
Tipo de operación: Sell
Número de características: 67
Total Ganadoras: 2311
Total Perdedoras: 1742
Se seleccionarán 1742 muestras por clase.
___________________________________


## Entrenamiento de los modelos

In [ ]:
# Train model functions
def train_classifier(X_train, y_train, model_type, n_features):
    # Definir algoritmo de validación cruzada
    cv = StratifiedKFold(n_splits=3)
    # Definir Pipeline
    pipeline = Pipeline([
            ('scaler', 'none'),
            ('reducer', 'none'),
            ('xgbclassifier', xgb.XGBClassifier(
                device = "cuda",
                verbosity=0)
            )
        ])
    # Definir mapa de estimadores
    estimator_map = {
        'scaler': {
            'standard': StandardScaler(),
            'robust': RobustScaler(),
            'none': 'passthrough'
        },
        'reducer': {
            'kernel_pca_rbf': KernelPCA(kernel='rbf'),
            'kernel_pca_linear': KernelPCA(kernel='linear'),
            'truncated_svd': TruncatedSVD(),
            'none': 'passthrough'
        }
    }
    # Definir espacio de hiperparámetros para compras
    param_grid = {
        'scaler': {'type': 'categorical', 'values': ['standard', 'robust']},
        'reducer': {'type': 'categorical', 'values': ['kernel_pca_rbf', 'kernel_pca_linear', 'truncated_svd']},
        'reducer__n_components': {'type': 'int', 'low': 2, 'high': n_features-1},
        'xgbclassifier__n_estimators': {'type': 'int', 'low': 50, 'high': 500},
        'xgbclassifier__max_depth': {'type': 'int', 'low': 3, 'high': 10}
    }
    try:
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search = GeneticAlgorithmCV(
            model_type=model_type,
            pipeline=pipeline,
            param_grid=param_grid,
            estimator_map=estimator_map,
            cv=cv,
            pop_size=20,
            generations=5,
            early_stopping_rounds=1,
            crossover_initial=0.9,
            crossover_end=0.1,
            mutation_initial=0.1,
            mutation_end=0.9,
            elitism=True,
            elite_size=3,
            tournament_size=3,
            n_random=10,
            n_jobs=-1,
            verbose=True,
        )
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en traing model {model_type}: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    print("####################################################################")
    print(f"Mejor puntuación de validación para {model_type}: {ga_search.best_score_}")
    print(f"Mejores parámetros encontrados para {model_type}: {ga_search.best_params_full_}")
    print("####################################################################")
    # Retornar mejor estimador
    return ga_search.best_estimator_

# Número de características totales
n_features = df.iloc[:, :-2].shape[1]
# Entrenar modelos en paralelo
with ThreadPoolExecutor(max_workers=2) as executor:
    print("Esperando que las tareas finalicen...")
    # enviar tareas de entrenamiento
    future_buy = executor.submit(train_classifier, X_buy, y_buy, "buy", n_features)
    future_sell = executor.submit(train_classifier, X_sell, y_sell, "sell", n_features)
    # esperar a que todas las tareas terminen
    futures = [future_buy, future_sell]
    wait(futures)
    # Obtener resultados una vez que ambas tareas han terminado
    model_buy = future_buy.result()
    model_sell = future_sell.result()
    print("¡Todas las tareas han terminado!")

Esperando que las tareas finalicen...


Generaciones buy:   0%|          | 0/5 [00:00<?, ?gen/s]

Generaciones sell:   0%|          | 0/5 [00:00<?, ?gen/s]

[1, sell] Fitness: 0.5200998407312034 | Best Fitness: 0.5200998407312034
[1, sell] Fitness Improvement: 0.0000 | Diversity: 0.8936
[1, sell] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 0.0000
[1, sell] Crossover Rate: 0.9000 | Mutation Rate: 0.1000
[1, buy] Fitness: 0.5131293418356568 | Best Fitness: 0.5131293418356568
[1, buy] Fitness Improvement: 0.0000 | Diversity: 0.8832
[1, buy] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 0.0000
[1, buy] Crossover Rate: 0.9000 | Mutation Rate: 0.1000
[2, sell] Fitness: 0.5307186664709781 | Best Fitness: 0.5307186664709781
[2, sell] Fitness Improvement: 0.0106 | Diversity: 0.9868
[2, sell] Normalized Fitness Improvement: 1.0000 | Normalized Diversity: 1.0000
[2, sell] Crossover Rate: 0.1954 | Mutation Rate: 0.8046
[2, buy] Fitness: 0.5159936458190187 | Best Fitness: 0.5159936458190187
[2, buy] Fitness Improvement: 0.0029 | Diversity: 0.8812
[2, buy] Normalized Fitness Improvement: 1.0000 | Normalized Diversity:

## Exportar modelos a formato ONNX

In [10]:
def save_onnx_model(mql5_files_folder, model, X, model_type):
    try:
        update_registered_converter(
            xgb.XGBClassifier,
            "XGBClassifier",
            calculate_linear_classifier_output_shapes,
            convert_xgboost,
            options={'nocl': [True, False], 'zipmap': [True, False, 'columns']}
        )
        model_onnx = convert_sklearn(
            model,
            f"pipeline_{model_type}_xgboost",
            [('input', FloatTensorType([None, X.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        with open(os.path.join(mql5_files_folder, f"model_{model_type}.onnx"), 'wb') as f:
            f.write(model_onnx.SerializeToString())
    except Exception as e:
        print(f"Error en exportar el modelo {model_type}: {e}")
        raise
    print(f"Modelo {model_type} ONNX exportado correctamente")

mql5_files_folder = r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files'
save_onnx_model(mql5_files_folder, model_buy, X_buy, "buy")
save_onnx_model(mql5_files_folder, model_sell, X_sell, "sell")

Modelo buy ONNX exportado correctamente
Modelo sell ONNX exportado correctamente


## Resultados

In [11]:
model_buy.steps

[('scaler', StandardScaler()),
 ('reducer', KernelPCA(kernel='rbf', n_components=62)),
 ('xgbclassifier',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device='cuda', early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=4, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=444, n_jobs=None,
                num_parallel_tree=None, random_state=None, ...))]

In [12]:
model_sell.steps

[('scaler', StandardScaler()),
 ('reducer', KernelPCA(n_components=34)),
 ('xgbclassifier',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device='cuda', early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=3, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=350, n_jobs=None,
                num_parallel_tree=None, random_state=None, ...))]

In [13]:
pd.DataFrame([model_buy.predict_proba(X_buy)[:10, 1], df['profit'][df['type']==1].head(10)])

,0,1,2,3,4,5,6,7,8,9
0,0.978855,0.994162,0.976274,0.96578,0.993179,0.930091,0.964636,0.959037,0.972032,0.979927
1,104.120000,72.580000,131.670000,60.42000,-61.750000,1592.100000,-1007.500000,-997.560000,-990.810000,-984.270000


In [14]:
pd.DataFrame([model_sell.predict_proba(X_sell)[:10, 1], df['profit'][df['type']==-1].head(10)])

,0,1,2,3,4,5,6,7,8,9
0,0.795706,0.874509,0.94741,0.812285,0.972795,0.875527,0.921102,0.611735,0.801093,0.934448
1,-870.600000,161.920000,2009.54000,-42.880000,-241.960000,923.130000,1031.680000,483.070000,986.000000,514.500000
